# TRAINING

In [13]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [14]:
from sagemaker.tensorflow import TensorFlow

hyperparams={'epochs'              : 1,
             'layers'              : 'heads', # heads or all
             'pretrained_weight'   : 'imagenet', # imagenet, coco
            }

bucket_name = sagemaker_session.default_bucket()
output_path = f's3://{bucket_name}/jobs'

In [15]:
import os

os.listdir("train/")

['.ipynb_checkpoints',
 'annotations.json',
 'requirements.txt',
 'Mask_RCNN',
 'model.py',
 'utils.py',
 'preprocessor.py',
 'train.py']

In [16]:
tf_estimator = TensorFlow(entry_point          = 'train.py', 
                          source_dir           = 'train/',
                          output_path          = f'{output_path}/',
                          code_location        = output_path,
                          role                 = role,
                          instance_count       = 1, 
                          instance_type        = 'ml.p3.2xlarge',
                          framework_version    = '1.15.3', 
                          py_version           = 'py3',
                          script_mode          = True,
                          sagemaker_session    = sagemaker_session,
                          hyperparameters      = hyperparams)

In [17]:
import time
job_name=f'tensorflow-maskrcnn-gpu-vehicle-parts-{time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())}'
tf_estimator.fit({'training'  : "s3://sagemaker-us-east-2-186402456289/sagemaker/mask_rcnn_localization_train",
                  'validation': "s3://sagemaker-us-east-2-186402456289/sagemaker/mask_rcnn_localization_valid",
                  }, 
                 job_name = job_name)

2021-08-25 00:46:57 Starting - Starting the training job...
2021-08-25 00:47:20 Starting - Launching requested ML instancesProfilerReport-1629852411: InProgress
......
2021-08-25 00:48:20 Starting - Preparing the instances for training......
2021-08-25 00:49:20 Downloading - Downloading input data
2021-08-25 00:49:20 Training - Downloading the training image........WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.

2021-08-25 00:50:34,742 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2021-08-25 00:50:36,420 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/usr/bin/python3 -m pip install -r requirements.txt

2021-08-25 00:50:40 Training - Training image download completed. Training in progress.Collecting IPython[all]
  Created wheel for pycocotools: filename=pycocoto

# DEPLOYMENT

In [19]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [20]:
from sagemaker.tensorflow.model import TensorFlowModel

In [29]:
artifact_dir = "s3://sagemaker-us-east-2-186402456289/jobs/tensorflow-maskrcnn-gpu-2021-08-25-00-46-51/output/model.tar.gz"

In [30]:
estimator = TensorFlowModel(model_data= artifact_dir, role=role, framework_version="1.15.3")

In [32]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

# PREDICTION

In [ ]:
import visualizer as visualize
import skimage

In [ ]:
import os
list_files = os.listdir("sample_files") 

In [ ]:
def detect(model,files):
    for file in files:
        image_path = f"sample_files/{file}"
        
        img = skimage.io.imread(image_path)
        img_arr = np.array(img)
        results = model.detect([img_arr], verbose=1)
        r = results[0]
        visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], 
                                    dataset_val.class_names, r['scores'])
        